#LSTM based neural model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd drive/My\ Drive/

/content/drive/My Drive


In [3]:
cd NLP_Assignment_2

/content/drive/My Drive/NLP_Assignment_2


In [4]:
import nltk
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
nltk.download('punkt')
nltk.download('stopwords')

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
file = open('speeches.txt',encoding="utf-8")
data = file.read()


In [0]:
def tkz(i):
    i = i.lower()
    tk = RegexpTokenizer(r'\w+')
    tokens = tk.tokenize(i)
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)

In [0]:
updated_inputs = tkz(data)

In [0]:
chars = sorted(list(set(updated_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [43]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 488088
Total vocab: 39


In [0]:
seq_length = 100
x_data = []
y_data = []

In [0]:
for i in range(0, input_len - seq_length, 1):
    seq1 = processed_inputs[i:i + seq_length]
    seq2 = processed_inputs[i + seq_length]
    x_data.append([char_to_num[char] for char in seq1])
    y_data.append(char_to_num[seq2])

In [46]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 487988


In [0]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [0]:
y = np_utils.to_categorical(y_data)


In [0]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [0]:
model = Sequential()
model.add(LSTM(256, input_shape=(xTrain.shape[1], xTrain.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [0]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])


In [0]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [0]:
model.fit(xTrain, yTrain, epochs=1, batch_size=256, callbacks=desired_callbacks)


In [0]:
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')


In [0]:
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [35]:
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" d getting elected going end would love support want money appreciate let take questions okay good go "


In [36]:
model.evaluate(xTest,yTest,verbose=0)

2.448194267593492

In [38]:
model.metrics_names

['loss']

References: stackabuse blog